In [3]:
import os
import json
from datetime import datetime, timedelta
from collections import defaultdict
import matplotlib.pyplot as plt

# Directory containing monthly watch history files
directory = "output_by_month"

# Path to the cycle data file
cycle_data_file = "cycle_data.json"

# Load cycle data
with open(cycle_data_file, "r") as file:
    cycle_data = json.load(file)

# Parse cycle start and end dates
cycles = [
    (
        datetime.strptime(cycle["period_start_date"], "%Y-%m-%d %H:%M:%S.%f"),
        datetime.strptime(cycle["period_end_date"], "%Y-%m-%d %H:%M:%S.%f")
    )
    for cycle in cycle_data
]

# Check if the directory exists
if not os.path.exists(directory):
    print(f"Directory '{directory}' not found. Please ensure it exists and contains the data files.")
else:
    # Iterate through all files in the directory
    for filename in os.listdir(directory):
        if filename.endswith(".json"):
            file_path = os.path.join(directory, filename)
            
            # Initialize dictionary to store total watch hours per day for the current month
            watch_time_per_day = defaultdict(float)
            
            # Read the JSON file
            with open(file_path, "r") as file:
                data = json.load(file)
            
            # Extract and process each video entry
            for entry in data.get("VideoList", []):
                watch_date = datetime.strptime(entry["Date"], "%Y-%m-%d %H:%M:%S")
                day = watch_date.date()  # Extract the date part
                watch_time_per_day[day] += 1 / 60  # Assuming each video is 1 minute

            # Extract data for plotting
            days = sorted(watch_time_per_day.keys())
            hours = [watch_time_per_day[day] for day in days]
            average_hours = sum(hours) / len(hours) if hours else 0

            # Determine cycle days within this month
            cycle_days = set()
            for start, end in cycles:
                for day in days:
                    if start.date() <= day <= end.date():
                        cycle_days.add(day)

            # Calculate average screentime during cycle periods
            cycle_hours = [watch_time_per_day[day] for day in days if day in cycle_days]
            cycle_average = sum(cycle_hours) / len(cycle_hours) if cycle_hours else 0

            # Plot the data
            plt.figure(figsize=(12, 7))
            
            # Bar chart for daily watch hours with cycle coloring
            bar_colors = ["red" if day in cycle_days else "skyblue" for day in days]
            plt.bar(days, hours, color=bar_colors, edgecolor='black', label='Daily Watch Hours')
            
            # Line chart for overall monthly average
            plt.axhline(y=average_hours, color='blue', linestyle='--', label=f'Monthly Average ({average_hours:.2f} hrs)')
            
            # Line chart for cycle average
            plt.axhline(y=cycle_average, color='green', linestyle='-', label=f'Cycle Average ({cycle_average:.2f} hrs)')
            
            # Customize plot
            plt.title(f"Watch Hours Per Day - {filename.split('.')[0]}", fontsize=18)
            plt.xlabel("Date", fontsize=14)
            plt.ylabel("Watch Hours", fontsize=14)
            plt.xticks(rotation=45, fontsize=10)
            plt.legend(fontsize=12)
            plt.tight_layout()

            # Save the plot
            chart_filename = f"{filename.split('.')[0]}_watch_hours_chart_with_cycle.png"
            plt.savefig(chart_filename)
            plt.close()

            print(f"Chart created and saved as {chart_filename}.")


Chart created and saved as 2024-05_watch_hours_chart_with_cycle.png.
Chart created and saved as 2024-08_watch_hours_chart_with_cycle.png.
Chart created and saved as 2024-09_watch_hours_chart_with_cycle.png.
Chart created and saved as 2024-11_watch_hours_chart_with_cycle.png.
Chart created and saved as 2024-07_watch_hours_chart_with_cycle.png.
Chart created and saved as 2024-06_watch_hours_chart_with_cycle.png.
Chart created and saved as 2024-10_watch_hours_chart_with_cycle.png.
